In [1]:
# import important library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow 
from keras.preprocessing import image
from keras.utils import to_categorical
import os
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder


## Model 1 - Basic CNNs

In [2]:
# Function to load images and labels into lists
def load_data():
    images = []
    labels = []
    for label in os.listdir('train'):
        label_path = os.path.join('train', label)
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

class CNN:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Conv2D(5, input_shape=(224,224,3), kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPool2D(2))
        self.model.add(Conv2D(15, kernel_size = 3, padding ='same', activation = 'relu'))
        self.model.add(Flatten())
        self.model.add(Dense(1, activation ='sigmoid'))                
        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    def fit(self,X,Y, epochs, batch_size):
        X = self.model.fit(X,Y, epochs = epochs, batch_size = batch_size)
        return X


In [3]:
train_img, train_labels = load_data()
encode = LabelEncoder()
train_labels = encode.fit_transform(train_labels)

cnn = CNN()
cnn.fit(train_img, train_labels, epochs = 10, batch_size = 5)

c:\Users\voqua\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5569 - loss: 104.0864
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7277 - loss: 1.8289
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9328 - loss: 0.2079
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9932 - loss: 0.0342
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9990 - loss: 0.0069
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0073
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0015
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 9.6205e-04
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 5.5941e-04
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 3.8714e-04


## Pretrained with ResNet50

In [34]:
# MODEL #2 - Pretrained model Resnet50
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
backbone = ResNet50(weights = 'imagenet')
backbone.summary()


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_6[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,636,712 (97.80 MB)

 Trainable params: 25,583,592 (97.59 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [36]:
for layer in backbone.layers:
    layer.trainable = False
class_layer = Dense(1, activation='sigmoid')

pt_model = Sequential([
    backbone,
    class_layer
])
pt_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [39]:
train2_img, train2_labels = load_data()
encode = LabelEncoder()
train2_labels = encode.fit_transform(train2_labels)

print(train2_img.shape, train2_labels.shape)
pt_model.fit(train2_img,train2_labels, epochs = 10, batch_size = 5)

(522, 224, 224, 3) (522,)
Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 26s 246ms/step - accuracy: 0.9321 - loss: 0.6549
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 26s 247ms/step - accuracy: 0.9423 - loss: 0.6376
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 26s 252ms/step - accuracy: 0.9478 - loss: 0.6222
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 27s 255ms/step - accuracy: 0.9421 - loss: 0.6067
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 28s 264ms/step - accuracy: 0.9515 - loss: 0.5909
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 28s 268ms/step - accuracy: 0.9539 - loss: 0.5813
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 28s 262ms/step - accuracy: 0.9570 - loss: 0.5634
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 27s 261ms/step - accuracy: 0.9560 - loss: 0.5451
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 30s 290ms/step - accuracy: 0.9517 - loss: 0.5379
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 28s 262ms/step - accuracy: 0.9664 - loss: 0.5133


In [49]:
from PIL import Image
image = Image.open('test1.jpg')
image_resized = image.resize((224, 224))
image_array = np.array(image_resized)
image_array_expanded = np.expand_dims(image_array, axis=0)
print(image_array_expanded.shape)  # This will print the shape of the array


(1, 224, 224, 3)


In [51]:
pred2 = pt_model.predict(image_array_expanded)
pred2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


array([[0.43875903]], dtype=float32)